# CNN resnet architecture using 2 diff datasets combined, classification part is changed by adding 3 extra layer to make more complex

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import cv2
import os

In [2]:
NUM_CLASSES = 2

# Fixed for Cats & Dogs color images
CHANNELS = 1

IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 100
BATCH_SIZE_VALIDATION = 100

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

In [3]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

/home/b160163ec/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/b160163ec/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/b160163ec/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/b160163ec/anaconda3/lib/python3.7/site-packages/tensorflow/python/f

In [4]:
resnet_weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels.h5'

In [5]:
model = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model.add(ResNet50(include_top = True, pooling = RESNET50_POOLING_AVERAGE, weights = resnet_weights_path))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model.add(Dense(500, activation = 'relu'))
model.add(Dense(250, activation = 'relu'))
model.add(Dense(250, activation = 'relu'))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))

# Say not to train first layer (ResNet) model as it is already trained
model.layers[0].trainable = False

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1000)              25636712  
_________________________________________________________________
dense (Dense)                (None, 500)               500500    
_________________________________________________________________
dense_1 (Dense)              (None, 250)               125250    
_________________________________________________________________
dense_2 (Dense)              (None, 250)               62750     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               25100     
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 202       
Total params: 26,350,514
Trainable params: 713,802
Non-trainable params: 25,636,712
______________________________________

In [7]:
from tensorflow.python.keras import optimizers

sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

In [8]:
DATADIR="/home/b160163ec/Documents/cnn_dataset/"
CATEGORIES=["0","2"]

In [9]:
training_data=[]

In [10]:
def create_training_data():
    count=0
    for category in CATEGORIES:
        path=os.path.join(DATADIR,category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(224,224))
                training_data.append([new_array,class_num])
                count=count+1
            except Exception as e:
                pass
    
create_training_data()
print(len(training_data))

9152


In [11]:
import random 

random.shuffle(training_data)
print("shuffling done...")

shuffling done...


In [12]:
training_data=np.asarray(training_data)
len(training_data[0])

2

In [13]:
training_data[0]

array([array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8),
       0], dtype=object)

In [14]:
x=[]
y=[]
for i in range(len(training_data)):
    if len(training_data[i][0].shape)==3:
        x.append(np.array(training_data[i][0]))
        y.append(training_data[i][1])

In [15]:
len(x)

9152

In [16]:
x=np.array(x).reshape(len(x),224,224,3)

In [17]:
x.shape

(9152, 224, 224, 3)

In [18]:
from keras.utils import to_categorical
y_binary = to_categorical(y)

Using TensorFlow backend.


In [19]:
y_binary

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [20]:
model.fit(x,y_binary,batch_size=10,epochs=20,validation_split=0.1)

Train on 8236 samples, validate on 916 samples
Epoch 1/20
8236/8236 [==============================] - 1724s 209ms/sample - loss: 0.5997 - acc: 0.7087 - val_loss: 0.5955 - val_acc: 0.7282
Epoch 2/20
8236/8236 [==============================] - 1716s 208ms/sample - loss: 0.5359 - acc: 0.7394 - val_loss: 0.6297 - val_acc: 0.7697
Epoch 3/20
8236/8236 [==============================] - 1702s 207ms/sample - loss: 0.3850 - acc: 0.8371 - val_loss: 0.8313 - val_acc: 0.7325
Epoch 4/20
8236/8236 [==============================] - 1699s 206ms/sample - loss: 0.3563 - acc: 0.8544 - val_loss: 0.5827 - val_acc: 0.8013
Epoch 5/20
8236/8236 [==============================] - 1702s 207ms/sample - loss: 0.3099 - acc: 0.8729 - val_loss: 0.5330 - val_acc: 0.8133
Epoch 6/20
8236/8236 [==============================] - 1696s 206ms/sample - loss: 0.2965 - acc: 0.8776 - val_loss: 0.3337 - val_acc: 0.8319
Epoch 7/20
8236/8236 [==============================] - 1699s 206ms/sample - loss: 0.3032 - acc: 0.8760 - v